In [145]:
import pandas as pd
import numpy as np
import nltk
import torch
nltk.download("stopwords")
nltk.download('punkt_tab')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from collections import Counter


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evergreen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/evergreen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [146]:
#print(torch.__version__)
#print("MPS available:", torch.backends.mps.is_available())
#print("MPS built:", torch.backends.mps.is_built())

In [147]:
#Testing Ground
file_object = open( "train.txt", "r" )
test_lines = file_object.readlines()[:2]
result = []
for comment in test_lines:
  comment = comment.strip()
  comment = "<s> " + comment + " </s>"
  print(comment)
    

# print(test_line)
# test_lines = sent_tokenize(test_line)
# test = Counter()
# print(test_lines)
# for line in test_lines:
#   line = "<s> " + line + " </s>"
#   print(line)
#   line = line.split()
#   for word in line:
#     test[word] += 1
# print(test)

<s> I booked two rooms four months in advance at the Talbott . We were placed on the top floor next to the elevators , which are used all night long . When speaking to the front desk , I was told that they were simply honoring my request for an upper floor , which I had requested for a better view . I am looking at a brick wall , and getting no sleep . He also told me that they had received complaints before from guests on the 16th floor , and were aware of the noise problem . Why then did they place us on this floor when the hotel is not totally booked ? A request for an upper floor does not constitute placing someone on the TOP floor and using that request to justify this . If you decide to stay here , request a room on a lower floor and away from the elevator ! I spoke at length when booking my two rooms about my preferences . This is simply poor treatment of a guest whom they believed would not complain . </s>
<s> I LOVED this hotel . The room was so chic and trendy , the bed was c

In [148]:
def collect_unigram(filename):
    unigram_counter = Counter()
    
    with open(filename, "r") as file_object:
        for comment in file_object.readlines(): # File is already document sentence tokenized
            comment = comment.strip() # Remove white space beginning/end
            comment = " <s> " + comment + " </s> " # Comment delimiter
            #print(comment)
            words = comment.split()  # Split into individual tokens
            for word in words:
                unigram_counter[word] += 1  # Count each word
    
    return unigram_counter

def collect_bigram(filename):
    bigram_counter = Counter()
    
    with open(filename, "r") as file_object:
        for comment in file_object.readlines():
            comment = comment.strip()
            comment = " <s> " + comment + " </s> "
            words = comment.split()
            
            for i in range(len(words) - 1): # Last one can't form a Tuple
                bigram = (words[i], words[i + 1])  # Tuple of two consecutive words
                bigram_counter[bigram] += 1
    
    return bigram_counter
unigram_counter = collect_unigram("train.txt") 
bigram_counter = collect_bigram("train.txt")

In [149]:
#Testing Ground


sum(unigram_counter.values())
#print(unigram_counter['I'])
print(bigram_counter)
#for (w1, w2), count in bigram_counter.items():
  #print(count)
print(bigram_counter[('.', 'The')])

Counter({('.', 'The'): 858, ('.', 'I'): 522, ('in', 'the'): 388, ('.', '</s>'): 371, ('of', 'the'): 342, ('.', 'We'): 339, (',', 'and'): 322, ('at', 'the'): 321, ('the', 'hotel'): 287, ('and', 'the'): 268, ('to', 'the'): 263, (',', 'but'): 243, (',', 'the'): 242, ('on', 'the'): 213, ('the', 'room'): 197, ('I', 'was'): 183, ('for', 'a'): 177, ('room', 'was'): 177, ('hotel', '.'): 173, ('.', 'It'): 172, ('this', 'hotel'): 168, (',', 'I'): 167, ('it', 'was'): 151, ('!', '!'): 147, ('for', 'the'): 146, ('was', 'very'): 145, ('from', 'the'): 137, ('and', 'I'): 134, ('was', 'a'): 133, ('had', 'a'): 129, ('with', 'a'): 126, ('a', 'great'): 119, ('we', 'were'): 115, ('in', 'a'): 115, ('to', 'be'): 115, ('is', 'a'): 110, ('<s>', 'I'): 109, ('I', 'had'): 107, ('in', 'Chicago'): 106, ('I', 'would'): 106, ('The', 'hotel'): 103, ('room', '.'): 102, ('hotel', 'is'): 101, ('stayed', 'at'): 100, ('front', 'desk'): 97, ('The', 'room'): 97, ('.', 'They'): 96, ('stay', 'at'): 95, (',', 'we'): 94, ('.', '

In [161]:
class NGramModel:
  def __init__(self, unigram_counts, bigram_counts=None):
    self.unigram_counts = unigram_counts
    self.bigram_counts = bigram_counts
    self.total_unigram_count = sum(self.unigram_counts.values())

    if bigram_counts: # If asking for Bigram Model
      self.bigram_context_counter = Counter()
      for (word_1, word_2), count in bigram_counts.items(): # item() format is Tuple(Tuple(word_1, word_2), number_appearance)
        self.bigram_context_counter[word_1] += count # This is the same as the unigarm_counts
      

  def unigram_probability(self, word):
    return self.unigram_counts[word] / self.total_unigram_count
  
  def bigram_probability(self, ask, given):
    if self.bigram_context_counter[given] == 0: 
      print("Unseen Before")
      return 0 # Unseen before
    else:
      # Return number of times { ask | Given} \over number of {Given}
      return self.bigram_counts[(given, ask)]/ self.bigram_context_counter[given]


In [151]:
tester = NGramModel(unigram_counter, bigram_counter)
print(tester.unigram_counts['I'], tester.bigram_context_counter["I"])

1659 1659


In [152]:
def handle_unknown_words(word_counts, threshold=5):
  vocab = set()
  unknown = 0

  for word, count in word_counts.items():
    if count > threshold:
      vocab.add(word)
    else:
      unknown += count
    
  #After the for loop, the total "Unknown"

In [163]:
example_bi = collect_bigram("example.txt")
example_uni = collect_unigram("example.txt")
example_tester = NGramModel(example_uni, example_bi)
print(example_bi)
print(example_tester.bigram_context_counter) #
print(example_tester.unigram_counts)
print(example_tester.unigram_probability("the"), example_tester.unigram_probability("like"))
print(example_tester.bigram_probability(ask="the", given="like"), example_tester.bigram_probability("students", "the")) # 1.0, 0.5


Counter({('<s>', 'the'): 1, ('the', 'students'): 1, ('students', 'like'): 1, ('like', 'the'): 1, ('the', 'assignment'): 1, ('assignment', '.'): 1, ('.', '</s>'): 1})
Counter({'the': 2, '<s>': 1, 'students': 1, 'like': 1, 'assignment': 1, '.': 1})
Counter({'the': 2, '<s>': 1, 'students': 1, 'like': 1, 'assignment': 1, '.': 1, '</s>': 1})
0.25 0.125
1.0 0.5
